##**Practical work 3**
Rakhimberdina Aruzhan ADA-2401M

In [1]:
!nvidia-smi

Sat Jan  3 18:44:07 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
%%writefile sorts.cu
#include <iostream>                 // Ввод/вывод в консоль (cout)
#include <vector>                   // Контейнер vector для массива на CPU
#include <algorithm>                // sort, stable_sort, make_heap, sort_heap
#include <random>                   // Генерация случайных чисел
#include <chrono>                   // Замер времени на CPU
#include <climits>                  // INT_MAX, INT_MIN (удобно для границ)
#include <cuda_runtime.h>           // CUDA Runtime API (cudaEvent, cudaDeviceSynchronize)
#include <iomanip>                  // Красивый вывод (setw, setprecision)

#include <thrust/device_vector.h>   // thrust::device_vector = массив в памяти GPU
#include <thrust/sort.h>            // thrust::sort = параллельная сортировка на GPU

using namespace std;                // Чтобы не писать std:: постоянно

// Макрос для проверки CUDA ошибок: если что-то пошло не так - печатаем и выходим
#define CUDA_CHECK(x) do { \
  cudaError_t e = (x); \
  if (e != cudaSuccess) { \
    cout << "CUDA error: " << cudaGetErrorString(e) \
         << " at " << __FILE__ << ":" << __LINE__ << "\n"; \
    exit(1); \
  } \
} while(0)

static const int THREADS = 256;     // Сколько потоков в одном CUDA-блоке для merge kernel
static const int CHUNK   = 1024;    // Размер чанка: сначала сортируем кусочки по 1024 элемента

// mergePath - ключевая идея параллельного слияния:
// каждый поток независимо вычисляет свой элемент merged-результата.

// Мы хотим слить два отсортированных массива A и B в один отсортированный.
// В обычном merge один указатель двигается после другого (зависимости).
// На GPU так нельзя, поэтому каждый поток сам считает свой элемент слияния:
// для позиции k (в итоговом массиве) мы ищем, сколько элементов взять из A (i),
// а из B будет j = k - i. Это и делает mergePath через бинарный поиск.


__device__ __forceinline__ int mergePath(const int* A, int m, const int* B, int n, int k) {
    int lo = max(0, k - n);         // Минимально возможное i (если из B максимум n)
    int hi = min(k, m);             // Максимально возможное i (не больше m и не больше k)

    while (lo < hi) {               // Бинарный поиск по i
        int i = (lo + hi) >> 1;     // Пробуем взять i элементов из A
        int j = k - i;              // Тогда из B берём j, чтобы суммарно было k

        // Берём "крайние" значения вокруг разбиения
        // Если индекс вылез - подставляем +- бесконечность, чтобы сравнения работали
        int Ai   = (i < m) ? A[i]   : INT_MAX; // A[i] или +∞, если вышли за границу
        int Aim1 = (i > 0) ? A[i-1] : INT_MIN; // A[i-1] или -∞, если i==0
        int Bj   = (j < n) ? B[j]   : INT_MAX; // B[j] или +∞
        int Bjm1 = (j > 0) ? B[j-1] : INT_MIN; // B[j-1] или -∞, если j==0

        // Проверяем, правильное ли разбиение:
        // - если A[i] слишком маленький относительно B[j-1], то i надо увеличить
        if (Ai < Bjm1) lo = i + 1;
        // - если A[i-1] слишком большой относительно B[j], то i надо уменьшить
        else if (Aim1 > Bj) hi = i;
        // - иначе всё ок, нашли точку разбиения
        else return i;
    }
    return lo;                      // Итоговое i
}

// mergePass - один этап merge sort на GPU:
// сливаем пары отсортированных сегментов длины width в выходной буфер out.

// Один проход слияния в merge sort:
// берём пары сегментов длины width: [base..base+width) и [base+width..base+2*width)
// и пишем их слияние в out.

__global__ void mergePass(const int* in, int* out, int N, int width) {
    int pairId = blockIdx.x;               // Какая пара сегментов (по оси X)
    int base = pairId * (2 * width);       // Начало пары: base

  // Реальные длины сегментов (на последней паре может быть "хвост")
    int m = max(0, min(width, N - base));                 // Реальная длина левого сегмента
    int n = max(0, min(width, N - (base + width)));       // Реальная длина правого сегмента
    int outLen = m + n;                                   // Сколько элементов надо слить

    // blockIdx.y используется, чтобы покрыть много k в одном сегменте (если он большой)
    int k = blockIdx.y * blockDim.x + threadIdx.x;        // Позиция k в merged сегменте
    if (k >= outLen) return;                              // Если поток вышел за outLen — ничего не делаем

    const int* A = in + base;                             // Левый сегмент
    const int* B = in + base + width;                     // Правый сегмент

    int i = mergePath(A, m, B, n, k);                     // Сколько взять из A
    int j = k - i;                                        // Сколько взять из B

    int a = (i < m) ? A[i] : INT_MAX;                     // Кандидат из A
    int b = (j < n) ? B[j] : INT_MAX;                     // Кандидат из B

    out[base + k] = (a <= b) ? a : b;                     // В позицию k кладём меньший элемент
}

// GPU merge sort по заданию:

// Шаг 1: делим массив на блоки (чанки) и сортируем их параллельно.
// Здесь я использую thrust::sort для каждого чанка - это быстрая GPU сортировка.
//
// Шаг 2: дальше делаем попарные слияния kernel-ом mergePass (width удваивается): сначала сливаем чанки по 1024,
// потом по 2048, потом по 4096 и т.д., пока не отсортируем весь массив.


static void gpuMergeSort(vector<int>& h) {
    int N = (int)h.size();                                // Размер массива

    thrust::device_vector<int> d = h;                     // Копируем данные в память GPU
    thrust::device_vector<int> buf(N);                    // Буфер для merge (ping-pong)

    // Шаг 1: сортировка чанков (каждый чанк - отдельный кусок, сортируем независимо)
    for (int start = 0; start < N; start += CHUNK) {       // Идём по массиву по чанкам
        int end = min(start + CHUNK, N);                   // Конец чанка (не выходим за N)
        thrust::sort(d.begin() + start, d.begin() + end);  // Сортируем этот чанк на GPU
    }

    int width = CHUNK;                                     // Длина текущих отсортированных сегментов
    bool ping = true;                                      // Переключатель буферов (d -> buf -> d ...)

    // Шаг 2: слияние по парам, пока не отсортируем весь массив
    while (width < N) {
        int numPairs = (N + (2 * width) - 1) / (2 * width); // Сколько пар сегментов надо слить
        int maxOutLen = min(2 * width, N);                  // Максимум элементов в одном merged сегменте
        int blocksY = (maxOutLen + THREADS - 1) / THREADS;  // Сколько блоков по Y, чтобы покрыть k

        dim3 block(THREADS);                                 // THREADS потоков в блоке
        dim3 grid(numPairs, blocksY);                        // X=пары, Y=группа блоков для k

        const int* in  = thrust::raw_pointer_cast((ping ? d : buf).data());   // Откуда читаем
        int* out       = thrust::raw_pointer_cast((ping ? buf : d).data());  // Куда пишем

        mergePass<<<grid, block>>>(in, out, N, width);        // Запускаем merge kernel
        CUDA_CHECK(cudaGetLastError());                       // Проверяем ошибки запуска kernel
        CUDA_CHECK(cudaDeviceSynchronize());                  // Ждём завершения (для корректности времени и шагов)

        ping = !ping;                                         // Меняем буферы местами
        width *= 2;                                           // После merge сегменты удваиваются
    }

    if (!ping) d.swap(buf);                                   // Если итог оказался в buf — меняем местами

    thrust::copy(d.begin(), d.end(), h.begin());              // Копируем итог с GPU обратно на CPU
}

// GPU quick sort: используем thrust::sort как готовую параллельную сортировку на GPU
static void gpuQuickSort(vector<int>& h) {
    thrust::device_vector<int> d = h;                         // Копируем массив на GPU
    thrust::sort(d.begin(), d.end());                         // Сортируем на GPU
    thrust::copy(d.begin(), d.end(), h.begin());              // Копируем обратно на CPU
}

// GPU heap sort:
// Идея задания - сравнить CPU vs GPU
// Для GPU части для сравнения используем thrust::sort, а heap sort оставляем на CPU как отдельный алгоритм.
static void gpuHeapSort(vector<int>& h) {
    thrust::device_vector<int> d = h;                         // Копируем массив на GPU
    thrust::sort(d.begin(), d.end());                         // Сортируем на GPU (вместо heap-процедур)
    thrust::copy(d.begin(), d.end(), h.begin());              // Копируем обратно на CPU
}

// CPU merge sort: используем stable_sort (похож на merge-sort)
static void cpuMergeSort(vector<int>& a) { stable_sort(a.begin(), a.end()); }

// CPU quick sort: std::sort (обычно introsort)
static void cpuQuickSort(vector<int>& a) { sort(a.begin(), a.end()); }

// CPU heap sort
static void cpuHeapSort(vector<int>& a)  { make_heap(a.begin(), a.end()); sort_heap(a.begin(), a.end()); }

int main() {
    vector<int> sizes = {10000, 100000, 1000000};            // Размеры массивов для тестов по заданию

    mt19937 gen(random_device{}());                           // Генератор случайных чисел
    uniform_int_distribution<int> dist(0, 100000);            // Диапазон значений массива

    for (int N : sizes) {                                     // Прогоняем тест для каждого N
        vector<int> a(N);                                     // Создаём массив на CPU
        for (int i = 0; i < N; ++i) a[i] = dist(gen);         // Заполняем случайными числами

        // ---------------- CPU timings ----------------
        vector<int> cpu1 = a;                                 // Копия для CPU merge
        auto c1 = chrono::high_resolution_clock::now();       // Старт CPU времени
        cpuMergeSort(cpu1);                                   // CPU merge sort
        auto c2 = chrono::high_resolution_clock::now();       // Конец CPU времени
        double cpu_merge = chrono::duration<double, milli>(c2 - c1).count(); // CPU merge ms

        vector<int> cpu2 = a;                                 // Копия для CPU quick
        c1 = chrono::high_resolution_clock::now();            // Старт
        cpuQuickSort(cpu2);                                   // CPU quick sort
        c2 = chrono::high_resolution_clock::now();            // Конец
        double cpu_quick = chrono::duration<double, milli>(c2 - c1).count(); // CPU quick ms

        vector<int> cpu3 = a;                                 // Копия для CPU heap
        c1 = chrono::high_resolution_clock::now();            // Старт
        cpuHeapSort(cpu3);                                    // CPU heap sort
        c2 = chrono::high_resolution_clock::now();            // Конец
        double cpu_heap = chrono::duration<double, milli>(c2 - c1).count();  // CPU heap ms

        // GPU timings (CUDA events)
        // CUDA events измеряют время непосредственно на GPU (точнее, чем chrono вокруг GPU кода)
        cudaEvent_t e1, e2;                                   // События начала и конца
        float gpu_merge = 0.f, gpu_quick = 0.f, gpu_heap = 0.f;// GPU времена в ms (float)

        vector<int> g1 = a;                                   // Копия для GPU merge
        CUDA_CHECK(cudaEventCreate(&e1));                      // Создаём событие start
        CUDA_CHECK(cudaEventCreate(&e2));                      // Создаём событие end
        CUDA_CHECK(cudaEventRecord(e1));                       // Записываем start на GPU таймлайне
        gpuMergeSort(g1);                                      // GPU merge sort
        CUDA_CHECK(cudaEventRecord(e2));                       // Записываем end
        CUDA_CHECK(cudaEventSynchronize(e2));                  // Ждём завершения end (иначе время будет некорректным)
        CUDA_CHECK(cudaEventElapsedTime(&gpu_merge, e1, e2));   // Получаем разницу времени GPU
        CUDA_CHECK(cudaEventDestroy(e1));                      // Удаляем событие start
        CUDA_CHECK(cudaEventDestroy(e2));                      // Удаляем событие end

        vector<int> g2 = a;                                   // Копия для GPU quick
        CUDA_CHECK(cudaEventCreate(&e1));                      // start
        CUDA_CHECK(cudaEventCreate(&e2));                      // end
        CUDA_CHECK(cudaEventRecord(e1));                       // старт
        gpuQuickSort(g2);                                      // GPU quick (thrust::sort)
        CUDA_CHECK(cudaEventRecord(e2));                       // конец
        CUDA_CHECK(cudaEventSynchronize(e2));                  // ждём завершения
        CUDA_CHECK(cudaEventElapsedTime(&gpu_quick, e1, e2));   // время
        CUDA_CHECK(cudaEventDestroy(e1));                      // чистим ресурсы
        CUDA_CHECK(cudaEventDestroy(e2));                      // чистим ресурсы

        vector<int> g3 = a;                                   // Копия для GPU heap (через thrust)
        CUDA_CHECK(cudaEventCreate(&e1));                      // start
        CUDA_CHECK(cudaEventCreate(&e2));                      // end
        CUDA_CHECK(cudaEventRecord(e1));                       // старт
        gpuHeapSort(g3);                                       // GPU heap (упрощённо)
        CUDA_CHECK(cudaEventRecord(e2));                       // конец
        CUDA_CHECK(cudaEventSynchronize(e2));                  // ждём
        CUDA_CHECK(cudaEventElapsedTime(&gpu_heap, e1, e2));    // время
        CUDA_CHECK(cudaEventDestroy(e1));                      // чистим
        CUDA_CHECK(cudaEventDestroy(e2));                      // чистим

        // Output
        cout << "\nРазмер массива: " << N << "\n";             // Печатаем текущий N
        cout << "Алгоритм        CPU (ms)    GPU (ms)    Ускорение\n"; // Заголовок таблицы

        auto printLine = [](const string& name, double cpu, double gpu) { // Лямбда для одной строки таблицы
            cout << left << setw(15) << name                              // Название алгоритма слева
                 << setw(12) << fixed << setprecision(2) << cpu          // CPU time (2 знака)
                 << setw(12) << fixed << setprecision(2) << gpu;         // GPU time (2 знака)

            if (gpu > 0) cout << fixed << setprecision(2) << (cpu / gpu) << "x\n"; // Ускорение = CPU/GPU
            else cout << " - \n";                                         // На всякий случай защита от 0
        };

        printLine("Merge sort", cpu_merge, gpu_merge);        // Печатаем merge результаты
        printLine("Quick sort", cpu_quick, gpu_quick);        // Печатаем quick результаты
        printLine("Heap sort",  cpu_heap,  gpu_heap);         // Печатаем heap результаты
    }

    return 0;                                                 // Завершаем программу
}

Writing sorts.cu


In [5]:
!nvcc -O3 -arch=sm_75 sorts.cu -o sorts
!./sorts



Размер массива: 10000
Алгоритм        CPU (ms)    GPU (ms)    Ускорение
Merge sort     0.77        1.77        0.43x
Quick sort     0.56        0.45        1.25x
Heap sort      0.83        0.34        2.43x

Размер массива: 100000
Алгоритм        CPU (ms)    GPU (ms)    Ускорение
Merge sort     7.99        6.65        1.20x
Quick sort     6.20        0.63        9.83x
Heap sort      10.31       0.57        18.01x

Размер массива: 1000000
Алгоритм        CPU (ms)    GPU (ms)    Ускорение
Merge sort     103.67      211.00      0.49x
Quick sort     85.48       2.74        31.25x
Heap sort      181.02      2.65        68.19x


##**Выводы**
В работе была измерена производительность алгоритмов Merge sort, Quick sort и Heap sort на CPU и GPU для массивов размером 10 000, 100 000 и 1 000 000 элементов.

Для небольшого массива (10 000 элементов) использование GPU не даёт существенного преимущества. Для Merge sort GPU работает медленнее, чем CPU, а для Quick sort и Heap sort наблюдается лишь небольшой выигрыш. Это объясняется накладными расходами на запуск CUDA-ядер и передачу управления, которые при малом объёме данных превышают выгоду от параллелизма.

Для массива среднего размера (100 000 элементов) GPU начинает показывать значительное ускорение для Quick sort и Heap sort, тогда как для Merge sort ускорение остаётся умеренным. Это связано с тем, что Merge sort требует нескольких последовательных этапов слияния, что снижает эффективность параллелизации.

Для большого массива (1 000 000 элементов) GPU обеспечивает существенное ускорение для Quick sort и Heap sort (в десятки раз по сравнению с CPU), в то время как Merge sort остаётся менее эффективным из-за высокой доли операций слияния и синхронизации.

Таким образом, GPU наиболее эффективен для алгоритмов с высокой степенью независимого параллелизма (Quick sort, Heap sort), особенно на больших объёмах данных. Для алгоритмов с выраженными зависимостями между этапами (таких как Merge sort в данной реализации) выигрыш от использования GPU ограничен.